In [ ]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
from datetime import datetime, timedelta

# Initialization

In [ ]:
reg_target = ['Île-de-France', 'Nouvelle-Aquitaine', 'Auvergne-Rhône-Alpes',
       'Bourgogne-Franche-Comté', 'Hauts-de-France', 'Grand Est',
       'Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte',
       'Centre-Val de Loire', 'Normandie', 'Pays de la Loire', 'Bretagne',
       'Occitanie', "Provence-Alpes-Côte d'Azur", 'Corse']

root = "/content/drive/MyDrive/Colab Notebooks/meteo_per_day/data/"

# Open Meteo Data file (test)

In [ ]:
monthdata = "2020-01"
dataset = pd.read_csv(root + monthdata + ".csv")
dataset = dataset.rename(columns={'Unnamed: 0': 'region'})
dataset.head()

,region,TempMax_Deg,TempMin_Deg,Wind_kmh,Wet_percent,Visibility_km,CloudCoverage_percent,Dayduration_hour,day
0,alsace,6,1.0,3,87,10.000,3,8:22:0,2020/01/01
1,aquitaine,12,10.0,12,89,10.000,65,8:52:0,2020/01/01
2,ardeche,10,4.0,6,81,10.000,2,8:52:0,2020/01/01
3,auvergne,8,3.0,4,83,10.000,13,8:44:0,2020/01/01
4,bourgogne,5,1.0,7,81,9.375,18,8:33:0,2020/01/01


In [ ]:
dataset

,region,TempMax_Deg,TempMin_Deg,Wind_kmh,Wet_percent,Visibility_km,CloudCoverage_percent,Dayduration_hour,day
0,alsace,6,1.0,3,87,10.000,3,8:22:0,2020/01/01
1,aquitaine,12,10.0,12,89,10.000,65,8:52:0,2020/01/01
2,ardeche,10,4.0,6,81,10.000,2,8:52:0,2020/01/01
3,auvergne,8,3.0,4,83,10.000,13,8:44:0,2020/01/01
4,bourgogne,5,1.0,7,81,9.375,18,8:33:0,2020/01/01
...,...,...,...,...,...,...,...,...,...
708,pays-de-la-loire,13,11.0,25,95,9.625,97,9:32:0,2020/01/31
709,picardie,12,8.0,22,89,9.625,96,9:16:0,2020/01/31
710,poitou-charentes,12,11.0,23,96,9.875,88,9:38:0,2020/01/31
711,rh-ne-alpes,13,9.0,5,92,8.875,82,9:40:0,2020/01/31


# New France Region mapping

The covid data collected refers to old regions !

In [ ]:
dataset['region'] = dataset['region'].map({'ile-de-france' : 'Île-de-France',
                                                   'limousin' : 'Nouvelle-Aquitaine',
                                                   'aquitaine' : 'Nouvelle-Aquitaine',
                                                   'poitou-charentes' : 'Nouvelle-Aquitaine',
                                                   '' : 'Auvergne-Rhône-Alpes',
                                                   'bourgogne' : 'Bourgogne-Franche-Comté',
                                                   'franche-comte' : 'Bourgogne-Franche-Comté',
                                                   'nord-pas-de-calais' : 'Hauts-de-France',
                                                   'picardie' : 'Hauts-de-France',
                                                   'alsace' : 'Grand Est',
                                                   'lorraine' : 'Grand Est',
                                                   '--' : 'Guadeloupe',
                                                   '--' : 'Martinique',
                                                   '--' : 'Guyane',
                                                   '--' : 'La Réunion',
                                                   '--' : 'Mayotte',
                                                   'centre' : 'Centre-Val de Loire',
                                                   'normandie' : 'Normandie',
                                                   'pays-de-la-loire' : 'Pays de la Loire',
                                                   'bretagne' : 'Bretagne',
                                                   'midi-pyrenees' : 'Occitanie',
                                                   'languedoc-roussillon' : 'Occitanie',
                                                   'provence-alpes-c-te-d-azur' : "Provence-Alpes-Côte d'Azur",
                                                   'corse' : 'Corse'
                                                  })

NameError: ignored

In [ ]:
dataset['region'].value_counts()

NameError: ignored

In [ ]:
dataset.head()

## Agregate on the regions (same date)

In [ ]:
dataset = dataset.groupby(['region', 'day'], dropna=True).mean()
dataset

In [ ]:
dataset.to_csv(root + "nr_" + monthdata + ".csv")

# Full Conversion function

In [ ]:
def convertRegionData(_month):
  dataset = pd.read_csv(root + _month + ".csv")
  dataset = dataset.rename(columns={'Unnamed: 0': 'region'})
  # Effectively do the region mapping
  dataset['region'] = dataset['region'].map({'ile-de-france' : 'Île-de-France',
                                                   'limousin' : 'Nouvelle-Aquitaine',
                                                   'aquitaine' : 'Nouvelle-Aquitaine',
                                                   'poitou-charentes' : 'Nouvelle-Aquitaine',
                                                   '' : 'Auvergne-Rhône-Alpes',
                                                   'bourgogne' : 'Bourgogne-Franche-Comté',
                                                   'franche-comte' : 'Bourgogne-Franche-Comté',
                                                   'nord-pas-de-calais' : 'Hauts-de-France',
                                                   'picardie' : 'Hauts-de-France',
                                                   'alsace' : 'Grand Est',
                                                   'lorraine' : 'Grand Est',
                                                   '--' : 'Guadeloupe',
                                                   '--' : 'Martinique',
                                                   '--' : 'Guyane',
                                                   '--' : 'La Réunion',
                                                   '--' : 'Mayotte',
                                                   'centre' : 'Centre-Val de Loire',
                                                   'normandie' : 'Normandie',
                                                   'pays-de-la-loire' : 'Pays de la Loire',
                                                   'bretagne' : 'Bretagne',
                                                   'midi-pyrenees' : 'Occitanie',
                                                   'languedoc-roussillon' : 'Occitanie',
                                                   'provence-alpes-c-te-d-azur' : "Provence-Alpes-Côte d'Azur",
                                                   'corse' : 'Corse'
                                                  })
  # Now need to group the identical days (coming from the same region) to ensure no duplicates
  dataset = dataset.groupby(['region', 'day'], dropna=True).mean()
  # Save the result
  dataset.to_csv(root + "nr_" + _month + ".csv")

## test function

In [ ]:
root = "/content/drive/MyDrive/Colab Notebooks/meteo_per_day/data/"
monthdata = "2020-02"
convertRegionData(monthdata)

# Convert a full year

In [18]:
year = "2020"
for month in range(12):
  smonth = year + "-" + str(month+1).rjust(2, '0')
  print ("Month: ", smonth)
  convertRegionData(smonth)

Month:  2020-01
Month:  2020-02
Month:  2020-03
Month:  2020-04
Month:  2020-05
Month:  2020-06
Month:  2020-07
Month:  2020-08
Month:  2020-09
Month:  2020-10
Month:  2020-11
Month:  2020-12
